In [1]:
import os
from tqdm import tqdm
# import feather
import pandas as pd
import numpy as np
from scipy import stats
import gc
from joblib import Parallel, delayed

In [2]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [3]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
df =pd.read_csv(fn)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,2562143,2562144,2562145,2562146,2562147,2562148,2562149,2562150,2562151,2562152
id,578,579,580,581,582,583,584,585,586,587,...,125396202,125396203,125396204,125396205,125396206,125396207,125396208,125396209,125396210,125396211
date,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,...,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15
store_nbr,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
item_nbr,103665,105574,105575,105577,105737,105857,106716,108696,108698,108701,...,2111684,2111870,2112404,2113914,2114566,2114812,2116416,2122188,2122676,2124052
unit_sales,2,8,15,2,2,12,2,3,6,3,...,7,21,5,1,1,1,1,1,2,1
onpromotion,Unk,Unk,Unk,Unk,Unk,Unk,Unk,Unk,Unk,Unk,...,False,False,True,False,False,True,False,False,False,False
item_family,BREAD/BAKERY,GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,DELI,DELI,DELI,...,BEVERAGES,PRODUCE,DAIRY,CLEANING,GROCERY I,CLEANING,GROCERY I,GROCERY I,BEVERAGES,"LIQUOR,WINE,BEER"
item_class,2712,1045,1045,1045,1044,1092,1032,2636,2644,2644,...,1120,2028,2108,3040,1034,3020,1060,1084,1124,1318
perishable,1,0,0,0,0,0,0,1,1,1,...,0,1,1,0,0,0,0,0,0,0
city,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,...,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito


In [6]:
df.h_type.value_counts()

Work Day      1766988
Weekend        596121
Holiday        103455
Event           88892
Additional       6697
Name: h_type, dtype: int64

In [30]:
# df[df.onpromotion == True]

In [31]:
# fn = '../cache/bkup/test7_t_store{}.csv'.format(2)
# df2 =pd.read_csv(fn)

In [32]:
df[(df.onpromotion == True) & (df.store_item_nbr == '1_105737')]

,id,date,store_nbr,item_nbr,onpromotion,item_family,item_class,perishable,city,state,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr


In [33]:
# len(set(df2.store_item_nbr))

In [34]:
# df2[df2.onpromotion == True]

In [35]:
# df2.h_type.value_counts()

In [36]:
# df2.h_desc.value_counts()

In [37]:
def days_from_next_promotion(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.onpromotion == True] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_promo'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_promo'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_promo'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_promo'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_promo'] = df_it['days_from_next_promo'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_promo_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_promo.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [38]:
def days_from_prev_promotion(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.onpromotion == True] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_promo'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_promo'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_promo'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_promo'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_promo'] = df_it['days_from_prev_promo'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_promo.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [39]:
def days_from_next_event(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Event'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_event'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_event'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_event'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_event'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_event'] = df_it['days_from_next_event'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_event_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_event.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [40]:
def days_from_prev_event(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Event'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_event'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_event'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_event'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_event'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_event'] = df_it['days_from_prev_event'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_event_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_event.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_event_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [41]:
def days_from_next_holiday(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Holiday'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_holiday'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_holiday'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_holiday'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_holiday'] = 0
                last_loc += 1
        fn_tmp = '../cache/train10_t_store{}_{}_next_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_holiday'] = df_it['days_from_next_holiday'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_holiday_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_next_holiday.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_next_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [42]:
def days_from_prev_holiday(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type == 'Holiday'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_holiday'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_holiday'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_holiday'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_holiday'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_holiday'] = df_it['days_from_prev_holiday'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_holiday_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_holiday.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_holiday_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [43]:
def days_from_next_nw(store_num):
    fn = '../cache/bkup/test7_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type != 'Work Day'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_nw'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_nw'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_nw'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_nw'] = 0
                last_loc += 1
        fn_tmp = '../cache/test7_t_store{}_{}_next_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_nw'] = df_it['days_from_next_nw'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_next_nw_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/test7_t_store{}_days_from_next_nw.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_next_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [44]:
def days_from_prev_nw(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_type != 'Work Day'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_nw'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_nw'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_nw'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_nw'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_nw'] = df_it['days_from_prev_nw'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_nw.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nw_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [45]:
def days_from_next_nn(store_num):
    fn = '../cache/bkup/test7_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_desc != 'Normal'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_next_nn'] = max_days
        last_loc = 0
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc < j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_next_nn'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_next_nn'] = diff
                last_loc += 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_next_nn'] = 0
                last_loc += 1
        fn_tmp = '../cache/test7_t_store{}_{}_next_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_next_nn'] = df_it['days_from_next_nn'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_next_nn_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/test7_t_store{}_days_from_next_nn.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_next_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [46]:
def days_from_prev_nn(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.h_desc != 'Normal'] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_nn'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_nn'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_nn'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_nn'] = 0
                last_loc -= 1
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_nn'] = df_it['days_from_prev_nn'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nn_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort('id')
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/train10_t_store{}_days_from_prev_nn.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/train10_t_store{}_{}_prev_nn_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [47]:
# days_from_next_promotion(df2)

In [48]:
# for i in tqdm(range(1,55)):
# #     days_from_next_promotion(i)
#     days_from_prev_promotion(i)
# #     days_from_next_event(i)
# #     days_from_prev_event(i)
# #     days_from_next_holiday(i)
# #     days_from_prev_holiday(i)
# #     days_from_next_nw(i)
# #     days_from_prev_nw(i)
# #     days_from_next_nn(i)
# #     days_from_prev_nn(i)

In [48]:
def days_from_prev_promotion_test(store_num):
    fn = '../cache/bkup/test7_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
    df['dts'] = df.date.map(np.datetime64)
    df_min_date = np.min(df.dts)
    df_max_date = np.min(df.dts)
    max_days = ((df_max_date - df_min_date))/np.timedelta64(1, 'D')
    n = 0
    items = list(set(df.store_item_nbr))
    for item in items:
        n = 1
        df_it = df[(df.store_item_nbr == item)].reset_index(drop=True)
        
        df_it['pe'] = 0 # promotion event
        df_it['pe'][df_it.onpromotion == True] = 1
        promo_indices = list(np.transpose(*np.where(df_it.pe)))
        
        df_it['days_from_prev_promo'] = max_days
        last_loc = len(df_it)-1
        for j in promo_indices:
            promo_date = df_it.loc[j, 'dts']
            while last_loc > j:
                curr_date = df_it.loc[last_loc, 'dts']
                if(curr_date == promo_date):
                    df_it.loc[last_loc, 'days_from_prev_promo'] = 0
                    continue
                diff = int((promo_date - curr_date)/ np.timedelta64(1, 'D'))
                df_it.loc[last_loc, 'days_from_prev_promo'] = abs(diff)
                last_loc -= 1    
            if last_loc == j:
                df_it.loc[last_loc, 'days_from_prev_promo'] = 0
                last_loc -= 1
        fn_tmp = '../cache/test7_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)
        df1 = pd.DataFrame()
        df1['id'] = df_it['id'].values
        df1['days_from_prev_promo'] = df_it['days_from_prev_promo'].values
        df1.to_csv(fn_tmp, float_format='%.5f', index=False)
        del df1
    df2 = pd.DataFrame()
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        df1 = pd.read_csv(fn_tmp)
        df2 = pd.concat([df2, df1], axis=0)
    df2 = df2.sort_values('id')
    df = df[(df.dts >= np.datetime64('2017-08-16')) & (df.dts <= np.datetime64('2017-08-31'))]
    min_id = df.iloc[0]['id']
    df2 = df2[df2.id >= min_id]
    assert(len(df) == len(df2))
    # assert(df.id == df2.id)
    s = sum(df2.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df2.isnull().sum())
    fn_tmp = '../cache/test7_t_store{}_days_from_prev_promo.csv'.format(store_num)
    delete_file_if_exists(fn_tmp)
    df2.to_csv(fn_tmp, float_format='%.5f', index=False)
    del df2
    del df
    for item in items:
        fn_tmp = '../cache/test7_t_store{}_{}_prev_promo_tmp.csv'.format(store_num, item)
        delete_file_if_exists(fn_tmp)

In [50]:
# for i in tqdm(range(1,55)):
#     days_from_next_promotion(i)
#     days_from_prev_promotion_test(i)
#     days_from_next_event(i)
#     days_from_prev_event(i)
#     days_from_next_holiday(i)
#     days_from_prev_holiday(i)
#     days_from_next_nw(i)
#     days_from_prev_nw(i)
#     days_from_next_nn(i)
#     days_from_prev_nn(i)

  0%|          | 0/54 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:45: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


0


  2%|▏         | 1/54 [04:23<3:52:37, 263.36s/it]

0


  6%|▌         | 3/54 [11:13<3:17:05, 231.87s/it]

...

0


 98%|█████████▊| 53/54 [2:15:24<02:23, 143.37s/it]

0


100%|██████████| 54/54 [2:17:47<00:00, 143.23s/it]

0


In [8]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
df =pd.read_csv(fn)
# df['dts'] = df.date.map(pd.to_datetime)
# df['dts1'] = df.date.map(np.datetime64)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
from datetime import date

In [10]:

df['year'] = df.dcount.map(lambda x: date.fromordinal(x).isocalendar()[0])
df['curr_week']  = df.dcount.map(lambda x: date.fromordinal(x).isocalendar()[1])

In [11]:
np.min(df.year)

2013

In [12]:
np.max(df.year)

2017

In [13]:
df['cw'] = df['curr_week'].astype('str') + '_' + df['year'].astype('str')

In [14]:
pd.to_datetime('2013-12-31').year

2013

In [15]:
pd.to_datetime('2013-12-31').isocalendar()

(2014, 1, 2)

In [16]:
import datetime

In [17]:
pd.to_datetime('2013-12-31').isocalendar()

(2014, 1, 2)

In [18]:
def get_prev_week(date):
    curr_year, curr_month, curr_day = str(date).split('-')
    
    today = datetime.date(int(curr_year),int(curr_month),int(curr_day))
    prev_week = today + datetime.timedelta(days=-7)
    selected_week = prev_week.isocalendar()[1]
    selected_year = prev_week.isocalendar()[0]
   
        
    prev_week = str(selected_week) + '_' + str(selected_year)
    return prev_week

In [19]:
def get_next_week(date):
    curr_year, curr_month, curr_day = str(date).split('-')
    
    today = datetime.date(int(curr_year),int(curr_month),int(curr_day))
    next_week = today + datetime.timedelta(days=7)
    selected_week = next_week.isocalendar()[1]
    selected_year = next_week.isocalendar()[0]
   
    next_week = str(selected_week) + '_' + str(selected_year)
    return next_week

In [20]:
def get_holidays_in_week(df, wk):
    weeks = list(np.unique(df[wk].values))
#     print(len(weeks))
    col = 'hols_in_'+ str(wk)
    df1 = pd.DataFrame()
    for week in tqdm(weeks):
        data = df[(df.h_type != 'Work Day') & (df.h_type != 'Weekend') & (df.cw == week)]
#         print(len(data))
        if len(data) > 0:
            df2 = pd.DataFrame()
            df2['id'] = data.id.values
            l = list(data.index.values)
            n = len(np.unique(data.date))
            df2[col] = n
            df1 = pd.concat([df1, df2], axis=0)
            del df2
    
    df1_id = df1.id.values
    df2_id = df.id[~df.id.isin(df1_id)]
    df2 = pd.DataFrame()
    df2['id'] = df2_id.values
    df2[col] = 0
    df1 = pd.concat([df1,df2], axis=0)
    df1 = df1.sort_values('id')
    del df2
#     print(len(df1), len(df))
    df[col] = df1[col].values
#     print(df.head())
    del df1
    return df

In [34]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,store_item_nbr,dts,year,curr_week,cw,pw,nw,hols_in_cw,hols_in_nw,hols_in_pw
0,578,2013-01-02,1,103665,2.0,Unk,BREAD/BAKERY,2712,1,Quito,...,1_103665,2013-01-02,2013,1,1_2013,52_2012,2_2013,0,0,0
1,579,2013-01-02,1,105574,8.0,Unk,GROCERY I,1045,0,Quito,...,1_105574,2013-01-02,2013,1,1_2013,52_2012,2_2013,0,0,0
2,580,2013-01-02,1,105575,15.0,Unk,GROCERY I,1045,0,Quito,...,1_105575,2013-01-02,2013,1,1_2013,52_2012,2_2013,0,0,0
3,581,2013-01-02,1,105577,2.0,Unk,GROCERY I,1045,0,Quito,...,1_105577,2013-01-02,2013,1,1_2013,52_2012,2_2013,0,0,0
4,582,2013-01-02,1,105737,2.0,Unk,GROCERY I,1044,0,Quito,...,1_105737,2013-01-02,2013,1,1_2013,52_2012,2_2013,0,0,0


In [22]:
df['pw']  = df['date'].map(lambda x: get_prev_week(x))

In [23]:
data = df[(df.h_type != 'Work Day') & (df.cw == '1_2013')]

In [24]:
len(data)

659

In [25]:
np.unique(data.date)

array(['2013-01-06'], dtype=object)

In [26]:
df['nw']  = df['date'].map(lambda x: get_next_week(x))

In [27]:
df['hols_in_cw'] = 0
df['hols_in_nw'] = 0
df['hols_in_pw'] = 0

In [28]:
%%time
df = get_holidays_in_week(df, 'cw')

100%|██████████| 242/242 [03:19<00:00,  1.21it/s]


CPU times: user 3min 24s, sys: 340 ms, total: 3min 25s
Wall time: 3min 25s


In [35]:
np.unique(df.hols_in_cw)

array([0, 1, 2, 3, 4, 5, 7])

In [30]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,2562143,2562144,2562145,2562146,2562147,2562148,2562149,2562150,2562151,2562152
id,578,579,580,581,582,583,584,585,586,587,...,125396202,125396203,125396204,125396205,125396206,125396207,125396208,125396209,125396210,125396211
date,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02,...,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15,2017-08-15
store_nbr,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
item_nbr,103665,105574,105575,105577,105737,105857,106716,108696,108698,108701,...,2111684,2111870,2112404,2113914,2114566,2114812,2116416,2122188,2122676,2124052
unit_sales,2,8,15,2,2,12,2,3,6,3,...,7,21,5,1,1,1,1,1,2,1
onpromotion,Unk,Unk,Unk,Unk,Unk,Unk,Unk,Unk,Unk,Unk,...,False,False,True,False,False,True,False,False,False,False
item_family,BREAD/BAKERY,GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,GROCERY I,DELI,DELI,DELI,...,BEVERAGES,PRODUCE,DAIRY,CLEANING,GROCERY I,CLEANING,GROCERY I,GROCERY I,BEVERAGES,"LIQUOR,WINE,BEER"
item_class,2712,1045,1045,1045,1044,1092,1032,2636,2644,2644,...,1120,2028,2108,3040,1034,3020,1060,1084,1124,1318
perishable,1,0,0,0,0,0,0,1,1,1,...,0,1,1,0,0,0,0,0,0,0
city,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,...,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito,Quito


In [36]:
set(df['hols_in_cw'].values)

{0, 1, 2, 3, 4, 5, 7}

In [32]:
def holidays_in_curr_prev_next_week(store_num):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(store_num)
    df =pd.read_csv(fn)
#     df['dts'] = df.date.map(pd.to_datetime)
    df['year'] = df.dcount.map(lambda x: date.fromordinal(x).isocalendar()[0])
    df['curr_week']  = df.dcount.map(lambda x: date.fromordinal(x).isocalendar()[1])
    df['cw'] = df['curr_week'].astype('str') + '_' + df['year'].astype('str')
    df['pw']  = df['date'].map(lambda x: get_prev_week(x))
    df['nw']  = df['date'].map(lambda x: get_next_week(x))
    df['hols_in_cw'] = 0
    df['hols_in_nw'] = 0
    df['hols_in_pw'] = 0
    df = get_holidays_in_week(df, 'cw')
    df = get_holidays_in_week(df, 'pw')
    df = get_holidays_in_week(df, 'nw')
    
    hols = ['hols_in_cw', 'hols_in_pw', 'hols_in_nw']
    df1 = df[hols]
    fn_tmp = '../cache/train10_t_store{}_cw_pw_nw_hols.csv'.format(store_num)
    print(fn_tmp)
    df1.to_csv(fn_tmp, float_format='%.5f', index=False)
    
    del df
    del df1

In [33]:
for i in range(1,55):
    holidays_in_curr_prev_next_week(i)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 242/242 [03:19<00:00,  1.21it/s]


../cache/train10_t_store1_cw_pw_nw_hols.csv


100%|██████████| 242/242 [02:36<00:00,  1.55it/s]


../cache/train10_t_store2_cw_pw_nw_hols.csv


100%|██████████| 242/242 [02:58<00:00,  1.36it/s]


...

../cache/train10_t_store52_cw_pw_nw_hols.csv


100%|██████████| 169/169 [01:11<00:00,  2.36it/s]


../cache/train10_t_store53_cw_pw_nw_hols.csv


100%|██████████| 242/242 [01:26<00:00,  2.79it/s]


../cache/train10_t_store54_cw_pw_nw_hols.csv
